In [19]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [20]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_lightning_3.tflite")
interpreter.allocate_tensors()

In [21]:
lowerjoints_list = [13,14,15,16]

In [22]:
for kp in joints_coordinates:
    ky, kx, kp_conf = kp
    print(int(ky), int(kx), kp_conf)

155 485 0.09142550826072693
159 486 0.044288456439971924
168 502 0.10068419575691223
132 483 0.020856738090515137
136 521 0.09888041019439697
57 484 0.05925583839416504
69 495 0.042238980531692505
300 245 0.05262744426727295
137 457 0.14167344570159912
363 407 0.25887399911880493
368 407 0.23643293976783752
75 501 0.02041211724281311
146 634 0.06537425518035889
70 472 0.2096032202243805
69 478 0.2031203806400299
369 396 0.2209474742412567
372 401 0.15641403198242188


In [23]:
def locating_joints(frame, keypoints, confidence_threshold):
    #y, x, c = frame.shape
    for i in lowerjoints_list:
        y, x, conf = joints_coordinates[i]
        if conf > confidence_threshold:
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

In [24]:
def knee_distance(frame,point):
   # y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[13]
    y2, x2,_ = joints_coordinates[14]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame,"Distance:" + str(distance), (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [25]:
def left_slope(frame,point):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[15]
  y2, x2,_ = joints_coordinates[13]
  m = (y2-y1)/(x2-x1)
  cv2.putText(frame,"left slope:" + str(m), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [26]:
def ankle_distance(frame,point):
    #y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[15]
    y2, x2,_ = joints_coordinates[16]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame," Ank:" + str(distance), (150,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [27]:
def right_slope(frame,point):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[16]
  y2,x2,_ = joints_coordinates[14]
  m2 = (y2-y1)/(x2-x1)
  cv2.putText(frame,"Right slope:" + str(m2), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [28]:
EDGES = {
    (13, 15): 'm',
    #(12, 14): 'c',
    (14, 16): 'c',
    (13, 14): 'c',
    (16,15) : 'm'
}

In [29]:
for edge, color in EDGES.items():
    p1, p2 = edge
    y1, x1, c1 = joints_coordinates[p1]
    y2, x2, c2 = joints_coordinates[p2]
    print((int(x2), int(y2)))

(396, 369)
(401, 372)
(478, 69)
(396, 369)


In [30]:
def join_joints(frame, keypoints, edges, confidence_threshold):
    #y, x, c = frame.shape
    print(keypoints.shape)

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = joints_coordinates[p1]
        y2, x2, c2 = joints_coordinates[p2]
        
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
           cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [31]:
def insoles(frame): 
  y1,x1 ,_ = joints_coordinates[13]
  y2,x2,_ = joints_coordinates[14] 
  y3,x3 ,_ = joints_coordinates[15]
  y4,x4,_ = joints_coordinates[16]
  import math 
  dis = (x2-x1)**2+(y2-y1)**2
  ankle_dis = (x4-x3)**2+(y4-y3)**2
  right_m= (y4-y2)/(x4-x2)
  left_m = (y3-y1)/(x3-x1)
  print("knee:", dis)
  print("ankle:", ankle_dis)
  print("left",left_m)
  print("Right",right_m)
#8739.635093408378
  
  #cv2.putText(frame,"insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  ankle = (x4-x3)**2+(y4-y3)**2
  #dis below 88 and ankle below 88, dis >80 and ankle > 80
  if (dis <= 7195.855191894665 and dis > 5608.360385571377 and ankle_dis <= 7195.855191894665 and ankle_dis > 5608.855051436771) or (dis < 7195.855191894665 and dis > 6495.855191894665 and ankle_dis >= 7195.855191894665 and ankle_dis <= 7795.855191894665 ):
     print("running loop 1")
     cv2.putText(frame,"Medium arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis <= 5608.855051436771 and ankle_dis <= 5608.855051436771) or (dis < 5608.855051436771 and ankle_dis > 5608.855051436771 and ankle_dis <= 5908.855051436771):
     #or (dis > 5608.855051436771 and ankle_dis < 7195.855191894665)
     print("running")
     cv2.putText(frame,"Low arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis >= 7195.855191894665 and ankle_dis >= 7195.855191894665):
     cv2.putText(frame,"High arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  else:
     cv2.putText(frame,"Other medical condition", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
     pass
            
       
        
        

        
    
    

In [32]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0),192 ,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(output_details)
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print("hello boy")
    print(keypoints_with_scores)                
    y, x, c = frame.shape
    print("gurs")
    print(np.multiply(keypoints_with_scores, [y,x,1]))
    print("Kaur")
    joints_coordinates = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    print(joints_coordinates)
    
    # Rendering
    locating_joints(frame, keypoints_with_scores, 0.4) 
    join_joints(frame, keypoints_with_scores, EDGES, 0.4)
    knee_distance(frame,keypoints_with_scores)
    ankle_distance(frame,keypoints_with_scores)
    left_slope(frame,keypoints_with_scores)
    right_slope(frame,keypoints_with_scores)
    insoles(frame)
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[{'name': 'StatefulPartitionedCall:0', 'index': 312, 'shape': array([ 1,  1, 17,  3]), 'shape_signature': array([ 1,  1, 17,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
hello boy
[[[[0.5380454  0.5431219  0.4890245 ]
   [0.47576663 0.5935091  0.563511  ]
   [0.48204464 0.4889427  0.5995538 ]
   [0.5027221  0.64816254 0.61213094]
   [0.51124406 0.42433342 0.7325685 ]
   [0.7715156  0.8167274  0.81589127]
   [0.7614174  0.28583127 0.8687899 ]
   [0.9600415  0.91554856 0.05878583]
   [0.9738097  0.2160095  0.01237014]
   [0.9791879  0.9861204  0.04025257]
   [0.8406853  0.4441031  0.00599292]
   [1.0164595  0.7805083  0.00901431]
   [0.26793897 0.45619974 0.01538026]
   [0.84027094 0.8731923  0.01543036]
   [0.77795506 0.2650765  0.01426399]
   [0.80110264 0.8017304  0.00168106]
   [0.79824466 0.51161516 0.00179

In [33]:
cap.release()
cv2.destroyAllWindows()